In [28]:
# import libraries
import rioxarray as rxr
from rasterio.enums import Resampling
from rasterio.warp import reproject
from scipy.interpolate import griddata
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
from pathlib import Path
import importlib.util

In [2]:
# load the setup.py module
path = Path.cwd().parent / 'src' / 'setup.py'
spec = importlib.util.spec_from_file_location('setup', path)
setup = importlib.util.module_from_spec(spec)
spec.loader.exec_module(setup)

# call the make_folders function
raw_data_dir, processed_data_dir, output_dir = setup.make_folders()

created directory data\raw_data\nDSM
created directory data\raw_data\orthomosaics
created directory data\processed_data
created directory data\metadata
created directory output


In [24]:
# data reading
#--------------
# define path to raw nDSM files
ndsm_path = raw_data_dir / 'nDSM'
ortho_path = raw_data_dir / 'orthomosaics'

# read one example file for both nDSM and orthomosaic
ndsm = rxr.open_rasterio(ndsm_path / 'nDSM_CFB030.tif', band_as_variable = True)
ortho = rxr.open_rasterio(ortho_path / 'CFB030_ortho.tif', band_as_variable = True)

print(ndsm)
print(ortho)

<xarray.Dataset>
Dimensions:      (x: 2600, y: 2600)
Coordinates:
  * x            (x) float64 4.01e+05 4.01e+05 4.01e+05 ... 4.011e+05 4.011e+05
  * y            (y) float64 5.281e+06 5.281e+06 ... 5.281e+06 5.281e+06
    spatial_ref  int32 0
Data variables:
    band_1       (y, x) float32 ...
Attributes:
    AREA_OR_POINT:  Area
<xarray.Dataset>
Dimensions:      (x: 10792, y: 10792)
Coordinates:
  * x            (x) float64 4.01e+05 4.01e+05 4.01e+05 ... 4.011e+05 4.011e+05
  * y            (y) float64 5.281e+06 5.281e+06 ... 5.281e+06 5.281e+06
    spatial_ref  int32 0
Data variables:
    band_1       (y, x) uint8 ...
    band_2       (y, x) uint8 ...
    band_3       (y, x) uint8 ...
    band_4       (y, x) uint8 ...
Attributes:
    AREA_OR_POINT:  Area


In [26]:
# assign NaN values to the nDSM
ndsm['band_1'] = ndsm['band_1'].where(ndsm['band_1'] != np.float32(-3.4e+38), np.nan)

# set _FillValue to NaN
fill_value_attrs = {'_FillValue': np.nan}
ndsm['band_1'].attrs.update(fill_value_attrs)
ndsm

<xarray.Dataset>
Dimensions:      (x: 2600, y: 2600)
Coordinates:
  * x            (x) float64 4.01e+05 4.01e+05 4.01e+05 ... 4.011e+05 4.011e+05
  * y            (y) float64 5.281e+06 5.281e+06 ... 5.281e+06 5.281e+06
    spatial_ref  int32 0
Data variables:
    band_1       (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    AREA_OR_POINT:  Area

In [29]:
# function to interpolate NaN values
def interpolate_nan(array):

    x, y = np.indices(array.shape)
    non_nan_elements = ~np.isnan(array)
    
    # coordinates and values of non-NaN elements
    known_x = x[non_nan_elements]
    known_y = y[non_nan_elements]
    known_values = array[non_nan_elements]
    
    # coordinates of NaN elements
    missing_x = x[np.isnan(array)]
    missing_y = y[np.isnan(array)]
    
    # interpolate and fill NaNs
    array[np.isnan(array)] = griddata(
        (known_x, known_y), known_values, (missing_x, missing_y), method='nearest'
    )
    
    return array

ndsm['band_1'].values = interpolate_nan(ndsm['band_1'].values)

In [33]:
ndsm.rio.to_raster(processed_data_dir / 'test.tif')

In [11]:
# downsampling of the orthomosaic
#---------------------------------
    
# calculate downscale factors
downscale_factor_x = ndsm.rio.width / ortho.rio.width
downscale_factor_y = ndsm.rio.height / ortho.rio.height
    
# calculate new heights and widths
x_downscaled = ortho.rio.width * downscale_factor_x
y_downscaled = ortho.rio.height * downscale_factor_y
    
# downsample the orthomosaic
ortho_downsampled = ortho.rio.reproject(
    ortho.rio.crs,
    shape=(int(y_downscaled), int(x_downscaled)),
    resampling=Resampling.bilinear,
    nodata=None
    )

In [33]:
print('old dimension ortho:', ortho.dims['x'])
print('new dimension ortho:', ortho_downsampled.dims['x'])

old dimension ortho: 10792
new dimension ortho: 2600
